In [1]:
import pandas as pd
import json
import os
from concurrent.futures import ProcessPoolExecutor
datadir = "/home/m/dev/ai/llm/wiki_en"
max_files = 32

file_list = [
    os.path.join(datadir, filename)
    for filename in os.listdir(datadir)
    if filename.endswith(".json")
][:max_files]
print(len(file_list))
def load_file(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
        df = pd.DataFrame(data)
    return df

with ProcessPoolExecutor() as executor:
    dfs = list(executor.map(load_file, file_list))

fulldata = pd.concat(dfs, ignore_index=True)

print(fulldata)

32
              id                                               text  \
0        9786730  Elizabeth Seton Academy may refer to: *Elizabe...   
1        9786749  __NOTOC__ is an editor and architecture critic...   
2        9786753  Northern Beaches Christian School is an indepe...   
3        9786755  Difference of Opinion was an Australian televi...   
4        9786756  Makoto Ueda may refer to: *Makoto Ueda (poetry...   
...          ...                                                ...   
357977  12454450  Donald Dee (August 9, 1943 – November 26, 2014...   
357978  12454459  Ardud (, Hungarian pronunciation: ; ) is a tow...   
357979  12454461  Corey James Miller (born October 25, 1968) is ...   
357980  12454477  Craig Robert Bradshaw (born 28 July 1983) is a...   
357981  12454491  The Allen Collins Band was a spinoff of southe...   

                                    title  
0                 Elizabeth Seton Academy  
1       Makoto Ueda (architecture critic)  
2       Nort

In [2]:
import torch
torch.cuda.empty_cache()
print(torch.cuda.get_device_name(0))
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import transformers
import math
from torch.optim.lr_scheduler import StepLR

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=4, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=1024, dropout=0.1):
        super(TransformerModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        
        self.output_layer = nn.Linear(d_model, vocab_size)
        self.d_model = d_model
        
    def forward(self, src, tgt,
                src_mask=None, tgt_mask=None, memory_mask=None,
                src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        # Embedding and positional encoding
        src = self.embedding(src) * math.sqrt(self.d_model)
        tgt = self.embedding(tgt) * math.sqrt(self.d_model)
        
        src = src.transpose(0, 1)
        tgt = tgt.transpose(0, 1)
        
        src = self.pos_encoder(src)
        tgt = self.pos_encoder(tgt)

        output = self.transformer(
            src, tgt,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_mask=memory_mask,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )
        
        output = output.transpose(0, 1)
        output = self.output_layer(output)
        return output

class WikiDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        

texts = fulldata['text'].tolist()

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size

model = TransformerModel(vocab_size)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.1)
dataset = WikiDataset(fulldata, tokenizer)
dataloader = DataLoader(dataset, batch_size=42, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
num_epochs = 4

num_batches = len(dataloader)
max_batches = num_batches

print(f"Number of batches: {num_batches}")

NVIDIA GeForce RTX 4080 SUPER


/home/m/dev/ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/m/dev/ai/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Number of batches: 8524


In [3]:
import time
import torch
from torch.amp import GradScaler, autocast

model.train()
total_estimated_time = 0
scaler = GradScaler()

for epoch in range(num_epochs):
    total_loss = 0.0
    epoch_start_time = time.time()

    for batch_idx, batch in enumerate(dataloader):
        if batch_idx >= max_batches:
            break

        if epoch == 0 and batch_idx == 10:
            average_batch_time = (time.time() - epoch_start_time) / 10
            estimated_remaining_batches = (num_epochs * max_batches) - 10
            total_estimated_time = average_batch_time * estimated_remaining_batches
            print(f"Estimated total remaining time: {total_estimated_time / 60:.2f} minutes.")

        batch_start_time = time.time()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        inputs = input_ids[:, :-1]
        targets = input_ids[:, 1:]

        src_key_padding_mask = (inputs == tokenizer.pad_token_id)
        tgt_key_padding_mask = src_key_padding_mask

        tgt_mask = model.transformer.generate_square_subsequent_mask(inputs.size(1)).to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(
                inputs, inputs,
                tgt_mask=tgt_mask,
                src_key_padding_mask=src_key_padding_mask,
                tgt_key_padding_mask=tgt_key_padding_mask
            )
            
            # Compute loss
            loss = criterion(outputs.view(-1, vocab_size), targets.contiguous().view(-1))

        scaler.scale(loss).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        scheduler.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        elapsed_batch_time = batch_end_time - batch_start_time
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{max_batches}, "
                  f"Loss: {loss.item():.4f}, Batch Time: {elapsed_batch_time:.2f} sec")

    avg_loss = total_loss / (batch_idx + 1)
    epoch_end_time = time.time()
    elapsed_epoch_time = epoch_end_time - epoch_start_time
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}, Epoch Time: {elapsed_epoch_time / 60:.2f} minutes")

print("Training completed.")

torch.save(model.state_dict(), 'wiki_transformer_model.pth')
print("Model saved as 'wiki_transformer_model.pth'")


/tmp/ipykernel_6562/1366603144.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Mixed precision scaling
/tmp/ipykernel_6562/1366603144.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/home/m/dev/ai/lib/python3.12/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch 1/4, Batch 1/8524, Loss: 10.4724, Batch Time: 0.32 sec
Estimated total remaining time: 184.73 minutes.
Epoch 1/4, Batch 11/8524, Loss: 9.0176, Batch Time: 0.13 sec
Epoch 1/4, Batch 21/8524, Loss: 8.1654, Batch Time: 0.13 sec
Epoch 1/4, Batch 31/8524, Loss: 7.7106, Batch Time: 0.13 sec
Epoch 1/4, Batch 41/8524, Loss: 7.4858, Batch Time: 0.13 sec
Epoch 1/4, Batch 51/8524, Loss: 7.4190, Batch Time: 0.13 sec
Epoch 1/4, Batch 61/8524, Loss: 7.2625, Batch Time: 0.13 sec
Epoch 1/4, Batch 71/8524, Loss: 7.3291, Batch Time: 0.13 sec
Epoch 1/4, Batch 81/8524, Loss: 7.1378, Batch Time: 0.13 sec
Epoch 1/4, Batch 91/8524, Loss: 7.1593, Batch Time: 0.13 sec
Epoch 1/4, Batch 101/8524, Loss: 7.2298, Batch Time: 0.13 sec
Epoch 1/4, Batch 111/8524, Loss: 7.1285, Batch Time: 0.13 sec
Epoch 1/4, Batch 121/8524, Loss: 7.3093, Batch Time: 0.13 sec
Epoch 1/4, Batch 131/8524, Loss: 7.2360, Batch Time: 0.13 sec
Epoch 1/4, Batch 141/8524, Loss: 7.3104, Batch Time: 0.13 sec
Epoch 1/4, Batch 151/8524, Loss:

In [7]:
def generate_text(prompt, max_length=20, temperature=0.6, top_k=50, repetition_penalty=1.2):
    model.eval()

    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    generated = input_ids.clone()
    for _ in range(max_length):
        inputs = generated[:, -max_length:]

        src_key_padding_mask = (inputs == tokenizer.pad_token_id).to(device)
        tgt_mask = model.transformer.generate_square_subsequent_mask(inputs.size(1)).to(device)

        with torch.no_grad():
            outputs = model(
                inputs, inputs,
                tgt_mask=tgt_mask,
                src_key_padding_mask=src_key_padding_mask,
                tgt_key_padding_mask=src_key_padding_mask
            )
            next_token_logits = outputs[:, -1, :] / temperature

            for token_id in set(generated.view(-1).tolist()):
                next_token_logits[:, token_id] /= repetition_penalty

            filtered_logits, indices = torch.topk(next_token_logits, top_k)
            probabilities = torch.softmax(filtered_logits, dim=-1)
            next_token = torch.multinomial(probabilities, num_samples=1)
            next_token = indices.gather(-1, next_token)

        generated = torch.cat([generated, next_token], dim=1)

        if next_token.item() == tokenizer.sep_token_id:
            break

    generated_text = tokenizer.decode(generated[0], skip_special_tokens=True)
    return generated_text



generate_text("The capital of france is")

'the capital of france is in the municipality located to the east in australia west state, within the province'